#**Ingesta a través de API rest**

#**Configuración de AWS CLI en Colab**


In [ ]:
##Instalación de boto3 & request para la API de los datos
!pip install boto3
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
import requests
import boto3
import json
from io import StringIO
import pandas as pd

In [ ]:
##Parametros de conexión con AWS###
!pip install awscli
!pip install --upgrade awscli

In [ ]:
session = boto3.Session(
     aws_access_key_id=aws_access_key_id,
     aws_secret_access_key=aws_secret_access_key,
     aws_session_token=aws_session_token
 )
s3 = session.client('s3')

In [ ]:
##Parametros del bucket##
s3_bucket='rawtrabajo1'
s3_target_file_name='rawtrabajo1/sea_lvls.csv'

#**Obteniendo la información del API que tiene el IMF**

####**Annual Surface Temperature Change**

In [ ]:
##Consumiendo la API
##Esta se encuentra en un formato GeoJSON
url_sea_lvls = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_3_3_melted_new/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

#Realizamos el request
response_sea_lvls = requests.get(url_sea_lvls)
data_sea_lvls= response_sea_lvls.json()

##Los archivos GeoJSON son tipo de estructura de datos que son frecuentemente utilizados para datos geográficos
##Estos consisten en varios elementos principales, con roles especificos.
##Para este caso estamos usando el elemento de "features" y "properties". El primero es un array utilizado para almacenar propiedades de los objetos
##El segundo contiene data no geométrica sobre las propiedades, aquí se encuentra la metadata

features_sea_lvls = data_sea_lvls['features']

##Normalizamos el JSON
df_sea_lvls= pd.json_normalize(features_sea_lvls, record_path=None, meta=['id', 'geometry'], errors='ignore')

##Lo almacenamos en un DataFrame de Pandas
df_sea_lvls = pd.DataFrame([feature.get('properties', {}) for feature in features_sea_lvls])

In [ ]:
##Vamos a convertir el DataFrame a CSV en memoria para el posterior almacenamiento en S3
csv_buffer = StringIO()
df_sea_lvls.to_csv(csv_buffer)

In [ ]:
df_sea_lvls.to_csv('sea_levels.csv', index=False)

In [ ]:
##Enviamos los datos a S3
s3.put_object(Body=csv_buffer.getvalue(),Bucket=s3_bucket,Key=s3_target_file_name)

ClientError: An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.